# 도커 Tip - 컨테이너의 파일 백업

도커 이미지나 컨테이너를 업데이트 하려고 컨테너를 삭제하면 기존에 컨테이너 안에 만들어 놓은 폴더나 파일은 사라진다. 

폴더와 파일을 호스트 컴퓨터에 백업해 놓으려면 다음 두 가지 방법 중 하나를 사용하면 된다.

1. `docker cp` 명령을 사용하여 복사한다.
2. 도커 실행시에 미리 shared data volume으로 지정하여 실행시킨다.

## `docker cp` 명령을 사용하여 복사하기

이 방법은 미리 shared data volume을 지정하지 않은 상태에서도 쓸 수 있다.

호스트 컴퓨터로 나온 다음 Docker Quickstart 셸에서 다음과 같이 명령한다.

```
$ docker cp 컨테이너_이름:컨테이너_폴더나_파일_이름 호스트컴퓨터의_폴더
```

예를 들어 rpython 이란 이름의 컨테이너 내에 있는 /home/dockeruser/notebook 폴더를 통째로 호스트 컴퓨터의 d:/ 라는 폴더 아래로 복사하고 싶으면 다음과 같이 명령한다.

```
$ docker cp rpython:/home/dockeruser/notebook d:/
```

만약 data 폴더 안에 untitled.ipynb 라는 파일만 있었으면 다음과 같이 notebook 폴더가 만들어지고 그 아래에 untitled.ipynb 파일이 있는 것을 볼 수 있다.

```
$ dir d:/notebook/
untitled.ipynb
```

만약 datascienceschool/rpython 이미지를 사용하고 있을 경우에는 /home/dockeruser 폴더를 통째로 복사하지 않도록 주의한다. 이 폴더 아래에는 Anaconda 설치 파일도 같이 있기 때문에 필요없이 많은 파이썬 실행 파일과 패키지 파일까지 복사된다.

이 방법의 단점은 아직까지 * 등의 wild card를 지원하지 않는다는 점이다. 따라서 정확한 파일이름이나 폴더 이름을 지정해야 한다.

반대로 호스트 컴퓨터에 백업해 놓은 파일이나 폴더를 컨테이너 안으로 복사하려면 다음과 같이 파일 인수를 바꾸면 된다.

```
$ docker cp d:/notebook rpython:/home/dockeruser/
```

다만 이 경우에는 호스트 컴퓨터의 사용자와 컨테이너의 사용자가 다른 경우 permission 오류가 발생할 수 있으므로 도커 컨테이너 안에서 다음과 같이 폴더 소유자를 변경해 주어야 한다.


```
$ docker attach rpython

dockeruser@bbbd63bfa054:~$ sudo chown dockeruser:dockeruser -R /home/dockeruser/notebook
```

## shared data volume 지정하여 실행하기

이 방법은 호스트 컴퓨터와 도커 컨테이간의 공유 폴더를 설정하는 방법이다. 다만 이 방법은 리눅스 호스트에서는 편리하나 윈도우즈/맥 호스트에서 Docker Toolbox와 가상 머신을 사용하는 경우에는 사용하기 힘들다. 공유 폴더가 boot2docker가 설치된 가상 머신과 컨테이너 사이에서만 공유되고 그것을 다시 윈도우즈/맥 호스트와 공유하려면 또 VirtualBox와 호스트 컴퓨터 간의 폴더 공유를 설정해야 하기 때문이다. 또한 이 방식은 최초로 docker run 명령을 사용하여 컨테이너를 실행하는 시점에서만 사용할 수 있고 이미 실행되어 있는 컨테이너에 공유 폴더를 추가할 수는 없다.

사용 방법은 다음과 같다.

`docker run` 명령을 사용할 때 `-v` 라는 옵션을 사용하면 호스트 컴퓨터의 특정 폴더를 컨테이너 이미지 안에 링크시켜서 컨테이너를 실행한다. 링크된 폴더안의 내용은 실제로는 호스트 컴퓨터에 바로 적용되며 해당 컨테이너를 지운 다음에도 변하지 않는다. 사용 방법은 다음과 같다.

```
docker run -v 호스트_폴더:컨테이너_폴더
```


만약 리눅스 호스트에서 datascienceschool/rpython 이미지를 rpython 이란 이름으로 실행시키면서 ~/rpython/notebook 이라는 폴더를 rpython 컨테이너 안의 /home/dockeruser/notebook 라는 폴더로 공유하고 싶다면 다음과 같이 도커 컨테이너를 생성한다. 

```
docker run -Pit --name rpython -p 8888:8888 -p 8787:8787 -p 6006:6006 -v ~/rpython/notebook:/home/dockeruser/notebook datascienceschool/rpython
```